<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/steinhaug_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

___Before you start:___   
_Version: v1.2_  
_Start by saving a copy to drive, and use the copy instead. This way you will be able to save your progress while running it. When you switch to the copy, make sure you are not running multiple sessions - under runtime select manage sessions and make sure you have killed all your sessions before you start running your saved copy._

This notebook is based on the [ShivamShrirao](https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth) and [TheLastBen](https://colab.research.google.com/github/TheLastBen/fast-stable-diffusion/blob/main/fast-DreamBooth.ipynb) Dreambooth colab.

NB! This requires custom editing for each run. If you are looking for a automated dreambooth check out this one, [Dreambooth Colab Edition - for people in a hurry](https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/Dreambooth_Colab_edition_for_people_in_a_hurry.ipynb).

In [ ]:
#@title Connect runtime and mount Google Drive
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

save_to_gdrive = True #@param {type:"boolean"}
if save_to_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
#@title Install dependencies
from IPython.display import clear_output
from pathlib import Path

!wget -q https://github.com/steinhaug/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py -O train_dreambooth_new.py
#!wget -q https://github.com/steinhaug/diffusers/raw/main/scripts/convert_original_stable_diffusion_to_diffusers.py
!wget https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_original_stable_diffusion_to_diffusers.py -O convert_original_stable_diffusion_to_diffusers.py
!wget -q https://github.com/steinhaug/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py

%pip install -qq git+https://github.com/steinhaug/diffusers
%pip install -q -U --pre triton
%pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers
%pip install omegaconf

clear_output()
print('[1;32mAll dependencies installed! ✓')

In [ ]:
#@title Login to HuggingFace 🤗
#@markdown You need to accept the model license before downloading or using the Stable Diffusion weights. Please, visit the [model card](https://huggingface.co/runwayml/stable-diffusion-v1-5), read the license and tick the checkbox if you agree. You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work.

!mkdir -p ~/.cache
!mkdir -p ~/.cache/huggingface
HUGGINGFACE_TOKEN = "" #@param {type:"string"}
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.cache/huggingface/token

clear_output()
print('[1;32mDone! ✓')


In [ ]:
#@title Select model and define folder for save
#@markdown If model weights should be saved directly in google drive (takes around 4-5 GB).
from IPython.display import clear_output;
save_to_gdrive = True #@param {type:"boolean"}

if save_to_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

#@markdown Name/Path of the initial model.
MODEL_NAME = "dreamlike-art/dreamlike-photoreal-2.0" #@param ["sd-dreambooth-library/disco-diffusion-style", "runwayml/stable-diffusion-v1-5", "dreamlike-art/dreamlike-photoreal-2.0", "SG161222/Realistic_Vision_V2.0", "Lykon/DreamShaper", "prompthero/openjourney"] {allow-input: true}
#@markdown Enter the directory name to save model at.
OUTPUT_DIR = "sd_weights/dreamlike-steinhaug-20" #@param {type:"string"}

if save_to_gdrive:
    OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR
else:
    OUTPUT_DIR = "/content/" + OUTPUT_DIR

!mkdir -p $OUTPUT_DIR

!mkdir /content/data/

import os
import shutil
from pathlib import Path

# - - -
# Copy n files from src to dst
# - - -
# Example: copy_nfiles(src,dst,500)
# Copies first 500 files from src to dst then stops.
def copy_nfiles(src, dst, max_n):
    path = Path(dst)
    if not path.exists():
        path.mkdir(parents = False, exist_ok = False)
    n = 0;
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isfile(s):
            shutil.copy2(s, d)
            n += 1
        if n == max_n:
            print(f'Copy complete, {max_n} files.')
            break

# - - -
# Copy n files from src into dst1, dst2 and dst3.
# - - -
# Example: copy_nfiles_to_dst123(src,dst1,dst2,dst3, 500)
# First 500 files from src into dst1, next 500 into dst2, next 500 into dst3 and stops.
def copy_nfiles_to_dst123(src, dst1, dst2, dst3, xsize):
    path = Path(dst1)
    if not path.exists():
        path.mkdir(parents = False, exist_ok = False)
    path = Path(dst2)
    if not path.exists():
        path.mkdir(parents = False, exist_ok = False)
    path = Path(dst3)
    if not path.exists():
        path.mkdir(parents = False, exist_ok = False)
    n = 0;
    for item in os.listdir(src):

        if n >= (xsize*2):
            dst = dst1
        elif n >= xsize:
            dst = dst2
        else:
            dst = dst3

        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isfile(s):
            shutil.copy2(s, d)
            n += 1
        if n == (xsize*3):
            print(f'Copy complete, 3x{xsize} files.')
            break

def ret_directoryFileCount(dir_path):
    file_count = 0
    for path in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, path)):
            file_count += 1
    return file_count


#copy_nfiles_to_dst123("/content/data/artwork_style_neg_text_v1-5_mse_vae_dpm2SaKarras50_cfg7_n4200", "/content/data/art1", "/content/data/art2", "/content/data/art3", 1000)
#copy_nfiles("/content/data/tmp/guy_v1-5_mse_vae_ddim50_cfg7_n4820", "/content/data/guy_images_500", 500)

saved_grid_count = 1

clear_output(); print('[1;32mDone! ✓')
print(f"[*] Weights will be saved at {OUTPUT_DIR}")

In [ ]:
#@title Regularization images - class images
%cd /content/data/
from zipfile import ZipFile

#@markdown This cell will download and prepare the regularization images needed for training. Just change the selected archive and reload this cell to change regularization images.
regularization_archive = "man-1000-ddim" #@param ["man-1000-ddim", "man-4820-ddim", "man-394-unsplash"]

REGZIP_ID = ''
if regularization_archive == 'man-1000-ddim':
    REGZIP_ID = "guy_v1-5_mse_vae_ddim50_cfg7_n1000"
    !wget https://huggingface.co/datasets/steinhaug/regularization/resolve/main/{REGZIP_ID}.zip -O {REGZIP_ID}.zip
elif regularization_archive == 'man-4820-ddim':
    REGZIP_ID = "guy_v1-5_mse_vae_ddim50_cfg7_n4820"
    !wget https://huggingface.co/datasets/ProGamerGov/StableDiffusion-v1-5-Regularization-Images/resolve/main/{REGZIP_ID}.zip -O {REGZIP_ID}.zip
elif regularization_archive == 'man-394-unsplash':
    REGZIP_ID = "man_unsplash-394"
    !wget https://huggingface.co/datasets/steinhaug/regularization/resolve/main/{REGZIP_ID}.zip -O {REGZIP_ID}.zip
else:
    raise Exception(f"[!] Unknown parameter for regularization_archive.")

if REGZIP_ID !== '':
    with ZipFile("/content/data/" + REGZIP_ID + ".zip", 'r') as zObject:
        zObject.extractall(path="/content/data")
    REGULARIZATION_DIR = "/content/data/" + REGZIP_ID
    clear_output()
    print('[1;32mRegularization images - unpacked and ready! ✓')

In [ ]:
# Concept setup

# 1 - 5
SET_NAME = 5

concepts_list = [
    {
        "instance_prompt":      "photo of steinhaug guy",
        "class_prompt":         "photo of a guy",
        "instance_data_dir":    "/content/data/steinhaug",
        "class_data_dir":       "/content/data/guy_images_auto",
    }
]

# Create the directory for training images and .json file 
import json; import os;
%cd /content/
for c in concepts_list:
    os.makedirs(c["instance_data_dir"], exist_ok=True)
with open("concepts_list.json", "w") as f:
    json.dump(concepts_list, f, indent=4)
clear_output(); print('[1;32mDone! ✓')
print(f"[*] Weights will be saved at {OUTPUT_DIR}")

In [ ]:
#@title Optional - file uploader for training images
#@markdown Use file manager on the left panel to upload (drag and drop) to each `instance_data_dir` (it uploads faster), or run this cell.

import os
from google.colab import files
import shutil

for c in concepts_list:
    print(f"Uploading instance images for `{c['instance_prompt']}`")
    uploaded = files.upload()
    for filename in uploaded.keys():
        dst_path = os.path.join(c['instance_data_dir'], filename)
        shutil.move(filename, dst_path)

In [ ]:
#@title Automatic training calibration
#@markdown We calibrate the training setting based on how many training images you have uploaded.<br>
#@markdown Regularization images are automatically added from the archive you selected previously.  

import os; import shutil;

for c in concepts_list:
    NUM_INSTANCE_IMAGES = ret_directoryFileCount(c["instance_data_dir"]);

if NUM_INSTANCE_IMAGES == 0:
    raise NameError('No training images uploaded, you need to do this first!')
else:
    print('[*] Training img count: ', NUM_INSTANCE_IMAGES)
    if SET_NAME == 2:
        # set2 - 990
        n_STEPS_IMG = 90
        n_CLASS_IMG = 15
        n_STEPS_WRP = 10
    elif SET_NAME == 3:
        # set3 - 900
        n_STEPS_IMG = 100
        n_CLASS_IMG = 15
        n_STEPS_WRP = 8
    elif SET_NAME == 4:
        # set4 - 808
        n_STEPS_IMG = 101
        n_CLASS_IMG = 14
        n_STEPS_WRP = 12
    elif SET_NAME == 5:
        # set5 - 
        n_STEPS_IMG = 80
        n_CLASS_IMG = 25
        n_STEPS_WRP = 10
    else:
        # set1 - 500, 640
        n_STEPS_IMG = 80
        n_CLASS_IMG = 12
        n_STEPS_WRP = 10

    LEARNING_RATE = 1e-6
    LR_SCHEDULE = "polynomial"
    NUM_CLASS_IMAGES = NUM_INSTANCE_IMAGES * n_CLASS_IMG
    MAX_NUM_STEPS = NUM_INSTANCE_IMAGES * n_STEPS_IMG
    LR_WARMUP_STEPS = int(MAX_NUM_STEPS / n_STEPS_WRP)
    MAX_TRAIN_STEPS = NUM_INSTANCE_IMAGES * 120

    print('[*] SET_NAME: ', SET_NAME)
    print(f"[*] NUM_CLASS_IMAGES {NUM_CLASS_IMAGES}")
    print(f"[*] MAX_NUM_STEPS {MAX_NUM_STEPS}")
    print(f"[*] LR_WARMUP_STEPS {LR_WARMUP_STEPS}")
    print(f"[*] MAX_TRAIN_STEPS {MAX_TRAIN_STEPS}")

    for c in concepts_list:
        if os.path.exists(c["class_data_dir"]):
            shutil.rmtree(c["class_data_dir"])
        os.makedirs(c["class_data_dir"], exist_ok=True)
        copy_nfiles(REGULARIZATION_DIR, c["class_data_dir"], NUM_CLASS_IMAGES)
        print('[*] Class_data_dir regularization imgs: ', ret_directoryFileCount(c["class_data_dir"]))

    print(f'Output: {OUTPUT_DIR}')


In [ ]:
#@title Dreambooth trainer - Lets train the model
!python3 train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$OUTPUT_DIR \
  --revision="main" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1337 \
  --resolution=512 \
  --train_text_encoder \
  --mixed_precision="no" \
  --use_8bit_adam \
  --gradient_checkpointing \
  --gradient_accumulation_steps=1 \
  --train_batch_size=1 \
  --learning_rate=$LEARNING_RATE \
  --lr_scheduler=$LR_SCHEDULE \
  --lr_warmup_steps=$LR_WARMUP_STEPS \
  --num_class_images=$NUM_CLASS_IMAGES \
  --max_train_steps=$MAX_TRAIN_STEPS \
  --save_interval=$n_STEPS_IMG \
  --sample_batch_size=4 \
  --save_sample_prompt="photo of steinhaug guy" \
  --n_save_sample=10 \
  --concepts_list="concepts_list.json"

clear_output()
print('[1;32mTraining completed! ✓')


# Cleanup and finnish

In [ ]:
#@title GRID MAKER: generate a grid of preview images from the saved checkpoints
import os
from sys import exit
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

OVERRIDE_OUTPUT_DIR = True #@param {type:"boolean"}

OVERRIDE_DIR = "/content/drive/MyDrive/sd_weights/dreamlike-steinhaug-20" #@param {type:"string"}

SAVE_GRID_IN_DIR = True #@param {type:"boolean"}

if OVERRIDE_OUTPUT_DIR:
    weights_folder = OVERRIDE_DIR
    breaki = 5
else:
    weights_folder = OUTPUT_DIR
    breaki = 99

folders = sorted([f for f in os.listdir(weights_folder) if f != "0" and os.path.isdir(weights_folder + '/' + f)], key=lambda x: int(x))

scale = 4

row = len(folders)
col = len(os.listdir(os.path.join(weights_folder, folders[0], "samples")))

if col > breaki:
    col = breaki

fig, axes = plt.subplots(row, col, figsize=(col*scale, row*scale), gridspec_kw={'hspace': 0, 'wspace': 0})

for i, folder in enumerate(folders):
    folder_path = os.path.join(weights_folder, folder)
    image_folder = os.path.join(folder_path, "samples")
    images = [f for f in os.listdir(image_folder)]
    for j, image in enumerate(images):
        if row == 1:
            currAxes = axes[j]
        else:
            currAxes = axes[i, j]
        if i == 0:
            currAxes.set_title(f"Image {j}")
        if j == 0:
            currAxes.text(-0.1, 0.5, folder, rotation=0, va='center', ha='center', transform=currAxes.transAxes)
        image_path = os.path.join(image_folder, image)
        img = mpimg.imread(image_path)
        currAxes.imshow(img, cmap='gray')
        currAxes.axis('off')
        if j == (breaki - 1):
            break;

if SAVE_GRID_IN_DIR:
    %cd $weights_folder
    plt.savefig(f'{saved_grid_count:04d}' + "_" + '_grid.png', dpi=72)

plt.tight_layout()
saved_grid_count += 1
%cd /content


In [ ]:
#@title GRID MAKER - Autodelete folders
import os

# todo auto select the folders

if os.path.isdir(weights_folder):
    user_inp_data = input('Do you want to continue? yes or no ')
    if user_inp_data == 'yes':
        keeper = "1040"
        keeper2 = "1111"

        do_delete_all = False

        folders = sorted([f for f in os.listdir(weights_folder) if f != "0" and os.path.isdir(weights_folder + '/' + f)], key=lambda x: int(x))

        for i, folder in enumerate(folders):
            if folder == keeper or folder == keeper2:
                do_delete_all = True

        for i, folder in enumerate(folders):
            print(folder)
            if folder == keeper or folder == keeper2:
                print('We keep this one!')
            else:
                dirPapth = f'{weights_folder}/' + folder
                if do_delete_all:
                    print(f'Deleting all files in {dirPapth}')
                    !rm -Rf {dirPapth}/scheduler/*
                    !rm -Rf {dirPapth}/text_encoder/*
                    !rm -Rf {dirPapth}/tokenizer/*
                    !rm -Rf {dirPapth}/unet/*
                    !rm -Rf {dirPapth}/vae/*
                    if os.path.isdir(dirPapth + '/args.json'):
                        !rm {dirPapth}/args.json
                    if os.path.isdir(dirPapth + '/model_index.json'):
                        !rm {dirPapth}/model_index.json

        print(f'Done!')
    else:
        print(f'Make sure you want to do this before continuing')


## CONVERTERS

In [ ]:
#@title CONVERT: diffusers to .ckpt
%cd /content/
#@markdown  Whether to convert to fp16, takes half the space (2GB).
fp16 = False #@param {type: "boolean"}

if fp16:
    ckpt_path = WEIGHTS_DIR + "/model_fp16.ckpt"
else:
    ckpt_path = WEIGHTS_DIR + "/model.ckpt"

half_arg = ""
if fp16:
    half_arg = "--half"
!python convert_diffusers_to_original_stable_diffusion.py --model_path $WEIGHTS_DIR  --checkpoint_path $ckpt_path $half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")

In [ ]:
#@title CONVERT: .ckpt to diffusers

PATH_CKPT = "/content/model/realisticVisionV13_v13.ckpt" #@param {type:"string"}
PATH_DIFFUSERS = "/content/diff_model" #@param {type:"string"}

## Display device type
#from tensorflow.python.client import device_lib
#temp = device_lib.list_local_devices()
#print(temp[0])

from pathlib import Path

path = Path(PATH_CKPT)
if not path.is_file():
  raise Exception(f"File not found! Path: {PATH_CKPT}")

path = Path(PATH_DIFFUSERS)
if not path.exists():
  print(f"[*] Create directory...")
  path.mkdir(parents = False, exist_ok = False)

%cd /content/
!python convert_original_stable_diffusion_to_diffusers.py --scheduler_type ddim --checkpoint_path $PATH_CKPT --image_size 512 --prediction_type epsilon --dump_path $PATH_DIFFUSERS --device cpu

WEIGHTS_DIR = PATH_DIFFUSERS

# Diffusion Inference mode: Lets load the model

In [ ]:
#@markdown Specify the weights directory to use (leave blank for latest)
WEIGHTS_DIR = "/content/drive/MyDrive/sd_weights/dreamlike-steinhaug-20/2814" #@param {type:"string"}
if WEIGHTS_DIR == "":
    from natsort import natsorted
    from glob import glob
    import os
    WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]

path = Path(WEIGHTS_DIR)
if not path.exists():
  raise Exception(f"[*] WEIGHTS_DIR does not exist!")
else:
  print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")


In [ ]:
#@title Load pipeline, init Diffusers - Stable diffusion inference
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

model_path = WEIGHTS_DIR             # If you want to use previously trained model saved in gdrive, replace this with the full path of model in gdrive

pipe = StableDiffusionPipeline.from_pretrained(model_path, safety_checker=None, torch_dtype=torch.float16).to("cuda")
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()
g_cuda = None

from IPython.display import clear_output; clear_output(); print('[1;32mDone! ✓')

In [ ]:
#@markdown Can set random seed here for reproducibility.
g_cuda = torch.Generator(device='cuda')
seed = 1 #@param {type:"number"}
g_cuda.manual_seed(seed)
saved_file_count = 1;

In [ ]:
# Place promt here 

# Mini Gradio - Inference

In [ ]:
#@markdown Run Gradio UI for generating images.
import gradio as gr

def inference(prompt, negative_prompt, num_samples, height=512, width=512, num_inference_steps=50, guidance_scale=7.5):
    with torch.autocast("cuda"), torch.inference_mode():
        return pipe(
                prompt, height=int(height), width=int(width),
                negative_prompt=negative_prompt,
                num_images_per_prompt=int(num_samples),
                num_inference_steps=int(num_inference_steps), guidance_scale=guidance_scale,
                generator=g_cuda
            ).images

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="Prompt", value="photo of zwx dog in a bucket")
            negative_prompt = gr.Textbox(label="Negative Prompt", value="")
            run = gr.Button(value="Generate")
            with gr.Row():
                num_samples = gr.Number(label="Number of Samples", value=4)
                guidance_scale = gr.Number(label="Guidance Scale", value=7.5)
            with gr.Row():
                height = gr.Number(label="Height", value=512)
                width = gr.Number(label="Width", value=512)
            num_inference_steps = gr.Slider(label="Steps", value=24)
        with gr.Column():
            gallery = gr.Gallery()

    run.click(inference, inputs=[prompt, negative_prompt, num_samples, height, width, num_inference_steps, guidance_scale], outputs=gallery)

demo.launch(debug=True)